In [1]:
open class Day(private val dayNum: Int) {
    val dataFolder = java.io.File("data21")
    fun readDay(number: Int) = dataFolder.resolve("day$number.txt").readText().lines()
    protected val input by lazy { readDay(dayNum) }
    
    open fun part1(): Int {
        return 0
    }
    
    open fun part2(): Int {
        return 0
    }
    
    fun render(): String {
        return """
            === DAY $dayNum ===
            Part 1 answer: ${part1()}
            Part 2 answer: ${part2()}
        """.trimIndent()
    }
}

fun day(number: Int, p1: (List<String>) -> Int, p2: (List<String>) -> Int): Day {
    return object : Day(number) {
        override fun part1(): Int {
            return p1(input)
        }
        override fun part2(): Int {
            return p2(input)
        }
    }
}

USE {
    render<Day> { it.render() }
}

In [2]:
object : Day(0) {
    override fun part1(): Int {
        return 0
    }
    
    override fun part2(): Int {
        return 0
    }
}

=== DAY 0 ===
Part 1 answer: 0
Part 2 answer: 0

In [3]:
object : Day(1) {
    val inputList = input.map { it.toInt() }
    fun List<Int>.howMany() = subList(1, size).zip(subList(0, size - 1)).count {(after, before) -> after > before}
    
    override fun part1(): Int {
        return inputList.howMany()
    }
    
    override fun part2(): Int {
        return inputList.windowed(3).map { it.sum() }.howMany()
    }
}

=== DAY 1 ===
Part 1 answer: 1616
Part 2 answer: 1645

In [4]:
object : Day(2) {
    override fun part1(): Int {
        var x = 0
        var d = 0

        input.forEach {
            val command = it.substringBefore(' ')
            val value = it.substringAfter(' ').toInt()
            when(command) {
                "up" -> d -= value
                "down" -> d += value
                "forward" -> x += value
            }
        }
        return x * d
    }
    
    override fun part2(): Int {
        var x = 0
        var d = 0
        var aim  = 0
        input.forEach {
            val command = it.substringBefore(' ')
            val value = it.substringAfter(' ').toInt()
            when(command) {
                "up" -> aim -= value
                "down" -> aim += value
                "forward" -> {
                    x += value
                    d += aim * value
                }
            }
        }
        return x * d
    }
}

=== DAY 2 ===
Part 1 answer: 1728414
Part 2 answer: 1765720035

In [5]:
object : Day(3) {
    val n = input.first().length
    val m = input.size
    
    fun List<Int>.convert() = fold(0) {acc, b -> acc * 2 + b}
    
    fun List<String>.myFilter(bitI: Int, mostCommon: Boolean): List<String> {
        val n = size
        val r = map { it[bitI] == '1' }
        val onesCount = r.count { it }
        val isOnes = onesCount * 2 >= n
        return filterIndexed { i, _ -> r[i] xor isOnes xor mostCommon  }
    }

    fun List<String>.myFold(mostCommon: Boolean) = foldIndexed(this) { bitI, acc, _ ->
        if (acc.size <= 1) acc else { acc.myFilter(bitI, mostCommon) }
    }.single().map {if(it == '0') 0 else 1}.convert()
    
    override fun part1(): Int {
        val result = input.fold(MutableList(n){0}) { acc, line ->
            line.forEachIndexed { i, c -> if (c == '1') acc[i]++ };
            acc 
        }.map { if (it * 2 >= m) 1 else 0 }
        val iresult = result.map { 1 - it }
        return result.convert() * iresult.convert()
    }
    
    override fun part2(): Int {
        return input.myFold(true) * input.myFold(false)
    }
}

=== DAY 3 ===
Part 1 answer: 2035764
Part 2 answer: 2817661

In [6]:
object : Day(4) {
    override fun part1(): Int {
        return 0
    }
    
    override fun part2(): Int {
        return 0
    }
}

=== DAY 4 ===
Part 1 answer: 0
Part 2 answer: 0